In [8]:
#//*** Collects pre-sorted Comments
#//*** Cleans Comments
#//*** Runs tfidf
#//*** Perform feature reduction on tfidf using Truncated SVD, preservering around 98% of variance. Cuts the Columns down from 200k+ to 6000
#//*** Saves the Truncated SVD for modeling
import os
import sys
import time
from datetime import date
from datetime import datetime
import time
import json
import platform

import stoneburner
#//*** Custom Functions:
#//*** mr_clean_text(input_series)
#//*** tokenize_series(input_series)
#//*** remove_stop_words(input_series)

# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#subreddits = ["wallstreetbets", "stocks", "wallstreetbetsOGs", "spacs", "investing", "pennystocks", "stockmarket", "options", "robinhoodpennystocks", "wallstreetbetsnew", "smallstreetbets"]
filepath = "./data/"
#filename_suffix = "_comments.csv.zip"
#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [9]:
#//*** Path to file aggregate
input_filename = "./data/processed_reddit_v4_1min.csv.zip"
input_filename = "./ignore_folder/processed_reddit_v4_1min_amc_stock_comments.pkl"
#output_filename = './ignore_folder/tsvd_model_ready_1min.csv.zip'

raw_df = pd.read_pickle(input_filename)
#//*** How many columns to pre
offset_target = "1"


raw_df

,time,open,high,low,close,volume,token
0,2019-07-24 09:00:00,4.09,4.090,4.0900,4.090,406,"[index, fund, tracks, performance, stock, mark..."
11,2019-07-24 09:35:00,4.14,4.150,4.1300,4.150,10116,"[combination, options, made, big, some, crmd, ..."
12,2019-07-24 09:36:00,4.14,4.170,4.1385,4.170,16449,"[idk, you, know, several, people, hybrids, new..."
13,2019-07-24 09:37:00,4.17,4.180,4.1540,4.175,15605,"[thanks, check, out]"
14,2019-07-24 09:40:00,4.19,4.195,4.1700,4.180,42314,"[im, waiting, monday, sell, covered, call, im,..."
...,...,...,...,...,...,...,...
4355042,2021-06-30 19:19:00,214.78,214.780,214.7800,214.780,100,"[always, possible, like, give, companies, cred..."
4355055,2021-06-30 19:40:00,214.80,214.800,214.8000,214.800,176,"[business, china, jaded, money, theres, one, t..."
4355064,2021-06-30 19:54:00,214.34,214.340,214.3200,214.320,811,"[automatically, downvoted, image, alone, then,..."
4355065,2021-06-30 19:55:00,214.15,214.150,214.0000,214.000,1499,"[how, get, 12, 000, lost, these, fools, cost, ..."


In [11]:
stocks_df = pd.read_csv("./stocks/amc_1min.csv.zip")
stocks_df.sort_values('time', inplace=True, ignore_index=True)
raw_df = pd.concat([raw_df,stocks_df])
raw_df.drop_duplicates(subset='time', keep=False,inplace=True)
raw_df.sort_values('time', inplace=True, ignore_index=True)

raw_df['token'] = raw_df['token'].fillna(" ")
#raw_df['tfidf'] = raw_df['token'].apply(lambda x: ' '.join(x))

#raw_df['token'] = raw_df['token'].replace(np.nan,"_____")

raw_df

,time,open,high,low,close,volume,token
0,2019-07-24 09:00:00,4.090000,4.090000,4.090000,4.090000,406,"[index, fund, tracks, performance, stock, mark..."
1,2019-07-24 09:35:00,9.968758,9.979624,9.951029,9.951029,1734,
2,2019-07-24 09:36:00,9.955795,10.037386,9.951029,10.037386,21448,
3,2019-07-24 09:37:00,10.055877,10.103536,10.055877,10.103536,22069,
4,2019-07-24 09:40:00,10.055877,10.084472,10.036814,10.065409,5081,
...,...,...,...,...,...,...,...
190321,2021-06-30 19:19:00,57.000000,57.030000,57.000000,57.030000,752,
190322,2021-06-30 19:40:00,56.950000,57.000000,56.950000,57.000000,6063,
190323,2021-06-30 19:54:00,57.140000,57.140000,57.010000,57.120000,3347,
190324,2021-06-30 19:55:00,57.100000,57.100000,57.020000,57.020000,1119,


In [4]:
#//*** Build Word Frequencies
word_freq = {}

for row in raw_df['token']:
    
    if row == "_____":
        continue
    
    for word in row:
        if word in word_freq.keys():
            word_freq[word] += 1
        else:
            word_freq[word] = 1

#//*** Sort Dictionary by values. Helpful for exploration
#marklist=sorted((value, key) for (key,value) in word_freq.items())
#sortdict=dict([(k,v) for v,k in marklist])

#//*** Generate a list of stop words
filter_words = ["_____"]

for word, freq in word_freq.items():
    if freq < 10:
        filter_words.append(word)
        
print("Total Unique Words: ",len(word_freq))     
print("Words used < 10 times: ",len(filter_words))


Total Unique Words:  93300
Words used < 10 times:  80103


In [18]:
marklist=sorted((value, key) for (key,value) in word_freq.items())
sortdict=dict([(k,v) for v,k in marklist])


print(sortdict)

{'+++': 1, '++msft': 1, '+100000': 1, '+101': 1, '+1011': 1, '+102': 1, '+10contracts': 1, '+10you': 1, '+110': 1, '+113': 1, '+1146': 1, '+116': 1, '+1200': 1, '+121': 1, '+122': 1, '+125': 1, '+1300': 1, '+140': 1, '+144': 1, '+1500': 1, '+1535': 1, '+160amtx': 1, '+164': 1, '+166': 1, '+16on': 1, '+170': 1, '+188': 1, '+19': 1, '+1k': 1, '+1x86': 1, '+1you': 1, '+1yr+2yr': 1, '+2040': 1, '+214': 1, '+233': 1, '+248': 1, '+250k': 1, '+25k': 1, '+2870': 1, '+288': 1, '+289': 1, '+300p': 1, '+301': 1, '+3060k': 1, '+308305': 1, '+32': 1, '+33': 1, '+34': 1, '+342': 1, '+36': 1, '+37': 1, '+38': 1, '+380c': 1, '+388': 1, '+39': 1, '+40mfinwhatever': 1, '+41': 1, '+420': 1, '+426': 1, '+450': 1, '+45intel': 1, '+47': 1, '+48': 1, '+4k': 1, '+500🧐': 1, '+50100': 1, '+50stock': 1, '+50€': 1, '+510': 1, '+57': 1, '+595': 1, '+5k': 1, '+5x': 1, '+61': 1, '+6115': 1, '+65': 1, '+679': 1, '+69': 1, '+700': 1, '+70k': 1, '+74': 1, '+77': 1, '+79': 1, '+7up': 1, '+82': 1, '+83i': 1, '+88': 1, '+

In [16]:
"""
#//*** Manually Remove stop words
#//*** Total Word Count
new_tokens = []
word_count = 0 
for row in raw_df['token']:
    if isinstance(row,list):
        word_count += len(row)
        for word in row:
            if word in filter_words:
                while word in filter_words:
                    row.remove(word)
        new_tokens.append(row)
                
    else:
        new_tokens.append([" "])
        
print(word_count)
"""
print()

In [5]:
#//************************************************
#//*** Clean text body
#//************************************************

#raw_df['token'] = stoneburner.remove_stop_words(stoneburner.tokenize_series(stoneburner.mr_clean_text(raw_df['body'],{"remove_empty":False})))
print("Done")

#//*** Detokenize the clean column as tfidf
raw_df['tfidf'] = raw_df['token'].apply(lambda x: ' '.join(x))
raw_df['tfidf'] = raw_df['tfidf'].fillna(" ")

raw_df

Done


,time,open,high,low,close,volume,token,tfidf
0,2019-07-24 09:00:00,4.090000,4.090000,4.090000,4.090000,406,"[index, fund, tracks, performance, stock, mark...",index fund tracks performance stock market ind...
1,2019-07-24 09:31:00,9.846181,9.912903,9.846181,9.912903,23448,,
2,2019-07-24 09:32:00,9.922435,9.951029,9.922435,9.922435,3394,,
3,2019-07-24 09:33:00,9.939591,9.998688,9.935302,9.998688,24294,,
4,2019-07-24 09:34:00,10.003453,10.017751,9.874776,9.960561,67195,,
...,...,...,...,...,...,...,...,...
127120,2021-07-12 19:56:00,41.250000,41.290000,41.220000,41.290000,10274,,
127121,2021-07-12 19:57:00,41.270000,41.350000,41.250000,41.330000,6832,,
127122,2021-07-12 19:58:00,41.330000,41.330000,41.250000,41.250000,14056,,
127123,2021-07-12 19:59:00,41.250000,41.280000,41.250000,41.250000,8974,,


In [17]:
#//*** Build tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#loop_list.append(tfidf.fit_transform(input_df['tfidf']))
tfidf_matrix = []
tfidf_list = []
tfidf = TfidfVectorizer(stop_words=filter_words)

print("Starting tfidf....")
start_time = time.time()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(raw_df['tfidf'])


print(f"Built: {round(time.time()-start_time,2)}")

print(tfidf_matrix)
print

Starting tfidf....
Built: 3.33
  (0, 42943)	0.08645982692297063
  (0, 58214)	0.06484176311387355
  (0, 16895)	0.05450158479469341
  (0, 66605)	0.04921336167961085
  (0, 9313)	0.056143522463136426
  (0, 80214)	0.061447444914441186
  (0, 72412)	0.045744118795754005
  (0, 80368)	0.11108900265285593
  (0, 63442)	0.05139792367006873
  (0, 55283)	0.04021192702210029
  (0, 73950)	0.04308839488466147
  (0, 51949)	0.048241159244066405
  (0, 58975)	0.05516826295068282
  (0, 88427)	0.03985201596470196
  (0, 50236)	0.03606811540234591
  (0, 34531)	0.05110591132175982
  (0, 30457)	0.08782319536469325
  (0, 75077)	0.05487514430866874
  (0, 29329)	0.063076567796982
  (0, 18174)	0.08090150884387814
  (0, 17839)	0.03777194974668722
  (0, 31587)	0.04001275119932962
  (0, 79192)	0.03928681055569845
  (0, 47575)	0.054721870840347075
  (0, 51252)	0.0457709243272105
  :	:
  (120796, 47162)	0.08484609046545274
  (120796, 27469)	0.0639508955749482
  (120796, 86317)	0.10009266311656319
  (120796, 78874)	0.0311

<function print>

In [7]:
tfidf

TfidfVectorizer()

In [ ]:
#//*** Remove Uneeded columns

#//*** Separate the time into a separate series
print("Removing uneeded columns...")
#//*** Remove uneeded columns
for col in ['body','token','tfidf']:
    if col in raw_df.columns:
        del raw_df[col]

raw_df


In [8]:
raw_df = pd.concat([raw_df,tfidf_matrix.toarray()],axis=1)

MemoryError: Unable to allocate 84.2 GiB for an array with shape (127125, 88881) and data type float64

In [ ]:
"""
#//*** Move this to modeling

input_filename = "./ignore_folder/processed_reddit_v4_60min.pkl.zip"
print("Building Target Offset Columns...")
#//*** Build the target variables and intervals of stock prices. This is a single value determined by target)offset

#//*** create a list of nan values of x length
nan_list = list(np.empty( offset_target )* np.nan )

#//*** Create target variable Price which is stocks + x columns in advance
#//*** Takes the closing price starting at x and gets the remainder, this generates the offset
#//*** nan_list fills the missing x values with nans
raw_df[f"close_{offset_target}"] = list(raw_df['close'][offset_target:]) + nan_list 
raw_df    
#start_time = time.time()
#print("Merging...")
##//*** Combine the tfidf sparse array with the stock values
#combined_df = pd.concat([raw_df,pd.DataFrame(tfidf_matrix.toarray())], axis=1)
#print (f"Done: {round(time.time()-start_time,2)}s")
"""
print()

In [ ]:
#training_filepath = "./ignore_folder/tsvd_model_ready_60min.csv.zip"
#stock_filepath =  "./ignore_folder/processed_reddit_v4_60min.pkl.zip"
offset_target = 1

start_time = time.time()
#print(f"Reading Stocks: {stock_filepath}")
#stock_df = pd.read_pickle(stock_filepath)

print(f"Building Time Series")
#//*** Peel off Time. Used for graphing
#time_series = tdf['time']

#print(f"Cleaning Columns...")
#//*** Remove uneeded columns
#for col in ['time','open','high','low','body']:
#    if col in tdf.columns:
#        del tdf[col]

print("Building Target Offset Columns...")
#//*** Build the target variables and intervals of stock prices. This is a single value determined by target)offset

#//*** create a list of nan values of x length
nan_list = list(np.empty( offset_target )* np.nan )

#//*** Create target variable Price which is stocks + x columns in advance
#//*** Takes the closing price starting at x and gets the remainder, this generates the offset
#//*** nan_list fills the missing x values with nans
#target = list(stock_df['close'][offset_target:]) + nan_list 

print(f"Check Stock Columns are aligned with offset and time")
#//*** Temporarily merge stock_df, time_series, and target series. 
#//*** Make sure everything is aligning correctly.
#temp_df = pd.concat([pd.DataFrame(time_series),pd.DataFrame(target),stock_df],axis=1)
#print(temp_df.head(10))
#print(temp_df.tail(10))

print(f"Building Combined df for training")
tdf = pd.concat([tdf,tfidf_matrix.toarray()],axis=1)

#//*** Delete unused variables to free up memory
try:
    
    del temp_df
    del stock_df
except:
    print()
print (f"Data is training ready: {round(time.time()-start_time,2)}s")

tdf

In [ ]:
#//*** Build TruncatedSVD

#from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import TruncatedSVD

start_time = time.time()
print(f"Begin Truncated SVD ")


#//*** Set the number of components to 6000. This is generating 98% variance capture
#//*** 60min data set took around 25minutes to build
tsvd = TruncatedSVD(6000)
#tsvd = TruncatedSVD(1000)
#tsvd = TruncatedSVD(1000)
tsvd_df = pd.DataFrame(tsvd.fit_transform(tfidf_matrix))
print(tsvd.explained_variance_ratio_.sum())

print (f"Truncated SVD Done: {round(time.time()-start_time,2)}s")

#//*** Write Truncated SVD to disk
tsvd_df.to_csv(output_filename, compression='zip', index=False)


In [ ]:
#//****1k PCA - .45
#//****100 PCA = .13
pca.explained_variance_ratio_.sum()

In [ ]:
import pickle 
#//*** Write the tsvd object to file, in case it's needed for additional analysis
outfile = open('./data/tsvd/truncatedSVD_60min.pkl.zip','wb')
pickle.dump(tsvd,outfile)
outfile.close()

In [ ]:
import sys
sys.getsizeof(combined_df)/1000000

In [ ]:
import time
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [ ]:
#//*** set the training columns to use everyting except time
predict_col = 'close_1'
train_cols = combined_df.columns[1:]
train_cols = ['close','volume']

x_train = combined_df[train_cols][:1000]
y_train = target_df[predict_col][:1000]
x_test = combined_df[train_cols][1001:1100]
y_test = target_df[predict_col][1001:1100]
x_train

In [ ]:
start_time = time.time()
print("Regressing")
regr = MLPRegressor(max_iter=500).fit(x_train,y_train)
#scores = cross_val_score(regr, x_train, y_train, cv=5)
#//*** Score the model
score = regr.score(x_train, y_train)
result = regr.predict(x_test)

mse = mean_squared_error(y_test, result)
r2 = r2_score(y_test,result)
#//*** Root Mean squared Error
rmse = sqrt(mse)
print(f"Complete: {round(time.time()-start_time,2)}" )

print(f"rmse: {rmse}")
print(f"r2: {r2}")

PCA Tutorial: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [ ]:
print(f"Begin Train Cols ")
train_cols = combined_df.columns[1:]
predict_col = 'close_1'
print(f"Begin Slice ")

x_train = combined_df[train_cols][:1000]
y_train = target_df[predict_col][:1000]
x_test = combined_df[train_cols][1001:1013]
y_test = target_df[predict_col][1001:1013]

start_time = time.time()
print(f"Begin PCA ")

# %%
from sklearn.decomposition import PCA

scaler = MinMaxScaler()

pca = PCA(n_components=3)
#X = pca.fit_transform( scaler.fit_transform(x_train) ,y_train)
x_train = pd.DataFrame(pca.fit_transform( x_train ,y_train))
x_test = pd.DataFrame(pca.fit_transform( x_test ,y_train))

print("Explained Variance Ratio")
for x in range(len(pca.explained_variance_ratio_)):
    print(x, pca.explained_variance_ratio_[x])
print (f"PCA Done: {round(time.time()-start_time,2)}s")



In [ ]:
start_time = time.time()
print("Regressing")
regr = MLPRegressor(max_iter=1000).fit(x_train,y_train)
#scores = cross_val_score(regr, x_train, y_train, cv=5)
#//*** Score the model
score = regr.score(x_train, y_train)
result = regr.predict(x_test)

mse = mean_squared_error(y_test, result)
r2 = r2_score(y_test,result)
#//*** Root Mean squared Error
rmse = sqrt(mse)
print(f"Complete: {round(time.time()-start_time,2)}" )

print(f"rmse: {rmse}")
print(f"r2: {r2}")

In [ ]:
display_size = 40

fig,ax = plt.subplots()
plot_x = np.arange(len(y_test))
ax.plot(plot_x,y_test )
ax.scatter(plot_x,result,color='red' )


    #plt.legend(loc='upper right',bbox_to_anchor=(1.35, 1.2))
plt.show()

In [ ]:
for x in range(len(pca.explained_variance_ratio_)):
    print(x, pca.explained_variance_ratio_[x])

In [ ]:
y_train
